In [1]:
import sys;sys.path.insert(0, '..')
from src.RMSE import BatchRMSE
import torch
import os
from src.FCN8 import FCN8
import src.core as core
import src.guassian as guassian


cwd = os.getcwd()
FCN8_model_dir = os.path.join(cwd, '..', 'models', 'FNC8_FINAL')

run_dir = os.path.join(FCN8_model_dir, 'run')
model_dir = os.path.join(FCN8_model_dir, 'model')


print(f"Using {core.TorchDevice} device")
torch.set_default_device(core.TorchDevice)

loaded_model = FCN8()
saved_model_path = model_path = os.path.join(model_dir, f'model_20240405_093356_53')
checkpoint = torch.load(saved_model_path, map_location=core.TorchDevice)
loaded_model.load_state_dict(checkpoint)
loaded_model.eval()

rmse = BatchRMSE()

predicted_heatmaps = None
single_sample = None
for i, sample in enumerate(core.TestDataSet):
    single_sample = sample
    image, heatmaps = sample
    image, heatmaps = image.to(core.TorchDevice), heatmaps.to(core.TorchDevice)

    ## We need to unsqueeze add the batch dimension to the image [1, 3, 96, 96]
    image = image.unsqueeze(0)
    # print("image.shape", image.shape)

    predicted_heatmaps = loaded_model(image)

    pred_heatmaps = predicted_heatmaps.cpu().detach()

    ## We need to remove the extra batch dimensions from the prediction [24, 96, 96]
    pred_heatmaps = pred_heatmaps.squeeze(0)

    # print("pred_heatmaps.shape", pred_heatmaps.shape)

    predicted_keypoints = guassian.heatmaps_to_keypoints_CoM(pred_heatmaps)

    ground_heatmaps = heatmaps.cpu().detach()

    gt_keypoints = guassian.heatmaps_to_keypoints_CoM(ground_heatmaps)

    rmse.add_pred_error(gt_keypoints, predicted_keypoints)


print(f"All Keypoints RMSE {rmse.get_all_keypoints_RMSE()}")



Using cpu device
All Keypoints RMSE 20.284266007201612


In [2]:
for i in range(24):
    try:
        print(f"All Keypoints RMSE {rmse.get_keypoint_RMSE(i)}")
    except:
        print(f"No RMSE for {i}th keypoint")

All Keypoints RMSE 23.36833946180064
All Keypoints RMSE 15.923941462605356
All Keypoints RMSE 14.541236288274119
All Keypoints RMSE 22.945440962898957
All Keypoints RMSE 21.569404576006672
All Keypoints RMSE 13.805777753809238
All Keypoints RMSE 40.3413951657741
All Keypoints RMSE 16.67928997819412
All Keypoints RMSE 14.119822248660576
All Keypoints RMSE 32.740275595929454
All Keypoints RMSE 18.105965522249694
All Keypoints RMSE 13.55759762127128
All Keypoints RMSE 15.190646515529064
All Keypoints RMSE 25.2806438656357
All Keypoints RMSE 13.914861924784365
All Keypoints RMSE 17.248747963772693
All Keypoints RMSE 11.952289101945565
All Keypoints RMSE 10.19911183125801
All Keypoints RMSE 21.17519349566745
All Keypoints RMSE 20.85111272029777
No RMSE for 20th keypoint
No RMSE for 21th keypoint
No RMSE for 22th keypoint
No RMSE for 23th keypoint
